# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
#Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 CombinedHeatPower_Turbogas.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 CombinedHeatPower_Turbogas.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

📁 CombinedHeatPower_Turbogas.ipynb executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\LCA4DHC


In [3]:
energysystem = pd.read_excel(energysystem_path, sheet_name = 'CHP-Turbogas')

# 2. Input parameters

In [4]:
Power = energysystem['Thermal capacity [kW]']; #Da file Excel
EfficiencyW = energysystem['Electrical efficiency based on LHV (between 0 and 1)']
EfficiencyQ = energysystem['Heat efficiency based on LHV (between 0 and 1)']
ElectricEnergyproduced = energysystem['Electric energy produced in a year (exported and consumed) [kWh /year]']
ThermalEnergyproduced_primary = energysystem['Thermal energy produced used [kWh /year]'] 
Qused = 1
SupplyTemperature = energysystem['Supply temperature [°C]']
ReturnTemperature = energysystem['Return temperature [°C]']
EnergyInputSourceGasBlend = energysystem['Energy input from gas-blend [kWh/year]']
EnergyInputSourceBiogas = energysystem['Energy input from biogas [kWh/year]']
EnergyInputSourceHydrogen = energysystem['Energy input from hydrogen [kWh/year]']

EquivalentHours = ThermalEnergyproduced_primary / Power

In [5]:
LHV = 36.8  # In MJ per Nm3 - Table 3.6 06_V_Natural Gas 3_3 Fuel Data
HHV = 40.4  # In MJ per Nm3 - Table 3.6 06_V_Natural Gas 3_3 Fuel Data

In [6]:
Lifespan = np.where(
    Power * (EfficiencyW / EfficiencyQ)  < 25,
    40000 / EquivalentHours,
    100000 / EquivalentHours
)

In [7]:
EfficiencyTot = EfficiencyW + EfficiencyQ
EnergyProducedLHV = (Power * Lifespan) /EfficiencyW
EnergyProducedHHV = EnergyProducedLHV * (HHV/LHV)
TotW = EnergyProducedLHV * EfficiencyW
TotQ = EnergyProducedLHV * EfficiencyQ * Qused

In [8]:
if Multifunctionality == 'Default': 
    ExternalTemperature = ExternalTemperature + 273.15 #K
    SupplyTemperature = SupplyTemperature + 273.15 #K
    ReturnTemperature = ReturnTemperature + 273.15 #K
    CarnotTemperature = (SupplyTemperature - ReturnTemperature) / np.log(SupplyTemperature / ReturnTemperature)
    ThermalEnergyproduced = EnergyProducedHHV
    ElectricEnergyproduced = (EnergyProducedHHV / EfficiencyQ) * EfficiencyW
    # AFQ calculation
    AFQ_GasTurbine = (ThermalEnergyproduced * (1 - (ExternalTemperature / CarnotTemperature))) / (ElectricEnergyproduced + ThermalEnergyproduced * (1 - (ExternalTemperature / CarnotTemperature)))
    AFQ_GasTurbine = np.array(AFQ_GasTurbine) 
    
    # AFW calculation
    AFW_GasTurbine = (1 - AFQ_GasTurbine)
    
else: 
    fwecr = 0.7
    fweexp = fwecr
    
    # Calculations
    InletEnergyCogenmode = EnergyProducedHHV * EfficiencyTot
    AFW_GasTurbine = (InletEnergyCogenmode * fwecr - (ElectricEnergyproduced * fweexp)) / EnergyProducedHHV 
    AFQ_GasTurbine = 1 - AFW_GasTurbine

if SupplyTemperature.eq(0).all() and ReturnTemperature.eq(0).all():
    AFQ_GasTurbine = np.zeros_like(AFQ_GasTurbine)
    AFW_GasTurbine = np.zeros_like(AFW_GasTurbine)

In [9]:
PowerEle = (Power / EfficiencyQ) * EfficiencyW 
RefPowerEle = 10000 #kW
RefMass = 114998.5 #kg
mass = PowerEle * (RefMass / RefPowerEle)

# 3. Life cycle inventories

## 3.1 Module A1-3

In [10]:
GasTurbine_concreteAT = ((0.22 * mass) / 2400) * AFQ_GasTurbine
GasTurbine_concreteCH = ((0.41 * mass) / 2400) * AFQ_GasTurbine
GasTurbine_copper = (0.03 * mass) * AFQ_GasTurbine
GasTurbine_diesel = (np.full(Power.shape, 759000)) * AFQ_GasTurbine
GasTurbine_electricity = (np.full(Power.shape, 46900)) * AFQ_GasTurbine
GasTurbine_heat = (np.full(Power.shape, 721050)) * AFQ_GasTurbine
GasTurbine_PEHD = (0.08 * mass) * AFQ_GasTurbine
GasTurbine_reinforcingsteel = (0.25 * mass) * AFQ_GasTurbine
GasTurbine_chromiumsteel = (0.01 * mass) * AFQ_GasTurbine

## 3.2 Module A4

In [11]:
GasTurbine_Truk_16_32 = ((500 + 200)*(mass)/1000) * AFQ_GasTurbine
GasTurbine_Light_Truk = (50*(mass)/1000) * AFQ_GasTurbine

## 3.3 Module B2

In [12]:
#y_Lifespan =  (EquivalentHours * RSP) / Lifespan
GasTurbine_LightTruk_B2 = (50 * (Lifespan/2) * 0.01 * (mass/1000))* AFQ_GasTurbine

## 3.4 Module B4

In [13]:
GasTurbineNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFGasTurbine = (RSP / ReqSL)  # Scaling factor

In [14]:
#Parameters for energy input source repartition
GasBlend = EnergyInputSourceGasBlend / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas)
Biogas = EnergyInputSourceBiogas / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas)
Hydrogen = EnergyInputSourceHydrogen / (EnergyInputSourceGasBlend + EnergyInputSourceBiogas + EnergyInputSourceHydrogen)


## 3.5 Module B6

In [15]:
x = 1
TotEnergyNeeds = np.array([EnergyProducedHHV])
TotEnergyNeeds = TotEnergyNeeds.flatten()
y = (TotEnergyNeeds * 3.6).flatten() * AFQ_GasTurbine

GasTurbine_EnergyVectorNG = (((1 / (HHV)) * 0.95) * A) * x * y * GasBlend
GasTurbine_EnergyVectorBiomethane = (((1 / (HHV)) * 0.042) * B) * x * y * GasBlend
GasTurbine_EnergyVectorHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * x * y) * C * GasBlend #120 MJ/kg
GasTurbine_EnergyVectorInputBiogas = ((1 / (HHV))) * x * y * Biogas
GasTurbine_EnergyVectorInputHydrogen = ((((1 / 0.9) * (1 / 3.6)) * 120) * x * y) * Hydrogen #120 MJ/kg
GasTurbine_WaterCooling = (0.00618146537053705 * x * y).flatten()
GasTurbine_Acetaldehyde = (0.000000001 * x * y).flatten()
GasTurbine_AceticAcid = (0.00000015 * x * y).flatten()
GasTurbine_Benzene = (0.00000039 * x * y).flatten()
GasTurbine_Benzoapyrene = (0.00000000001 * x * y).flatten()
GasTurbine_Butane = (0.00000068 * x * y).flatten()
GasTurbine_CarbonDioxideFossil = (0.05406 * x * y).flatten()
GasTurbine_CarbonMonoxideFossil = (0.000003 * x * y).flatten()
GasTurbine_DinitrogenMonoxide = (0.00000048 * x * y).flatten()
GasTurbine_Formaldehyde = (0.00000010 * x * y).flatten()
GasTurbine_Mercury = (0.00000000003 * x * y).flatten()
GasTurbine_MethaneFossil = (0.00000193 * x * y).flatten()
GasTurbine_NitrogenOxides = (0.00001931 * x * y).flatten()
GasTurbine_PAH = (0.00000001 * x * y).flatten()
GasTurbine_Particulates = (0.00000010 * x * y).flatten()
GasTurbine_Pentane = (0.00000116 * x * y).flatten()
GasTurbine_Propane = (0.00000019 * x * y).flatten()
GasTurbine_PropionicAcid = (0.00000002 * x * y).flatten()
GasTurbine_SulfurDioxide = (0.00000048 * x * y).flatten()
GasTurbine_Toluene = (0.00000019 * x * y).flatten()
GasTurbine_WaterAir = (0.0000729530701641572 * x * y).flatten()
GasTurbine_WaterWater = (0.00610851230037289 * x * y).flatten()

## 3.6 Module C2

In [16]:
GasTurbine_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.7 Module C3

In [17]:
GasTurbine_EOL_concreteAT = GasTurbine_concreteAT
GasTurbine_EOL_concreteCH = GasTurbine_concreteCH
GasTurbine_EOL_copper = GasTurbine_copper
GasTurbine_EOL_PEHD = GasTurbine_PEHD
GasTurbine_EOL_reinforcingsteel = GasTurbine_reinforcingsteel
GasTurbine_EOL_chromiumsteel = GasTurbine_chromiumsteel

## 3.8 Module C4

In [18]:
GasTurbine_EOL_landfill_concreteAT = GasTurbine_EOL_concreteAT * LandfillConcrete
GasTurbine_EOL_landfill_concreteCH = GasTurbine_EOL_concreteCH * LandfillConcrete
GasTurbine_EOL_landfill_copper = GasTurbine_EOL_copper * LandfillMetals
GasTurbine_EOL_inc_PEHD = GasTurbine_EOL_PEHD * IncinerationPEHD
GasTurbine_EOL_landfill_PEHD = GasTurbine_EOL_PEHD * LandfillPEHD
GasTurbine_EOL_landfill_reinforcingsteel = GasTurbine_EOL_reinforcingsteel * LandfillMetals
GasTurbine_EOL_landfill_chromiumsteel = GasTurbine_EOL_chromiumsteel * LandfillMetals2

## 3.9 Module D1 (related with the export of secondary materials)

In [19]:
GasTurbine_EOL_recycling_concreteAT = (RecyclingConcrete * RecyclingEfficiencyConcrete * GasTurbine_EOL_concreteAT) - (RecycledContentConcrete * GasTurbine_EOL_concreteAT)
GasTurbine_EOL_primary_concreteAT = ((RecycledContentConcrete * GasTurbine_EOL_concreteAT) - (RecyclingConcrete * RecyclingEfficiencyConcrete * GasTurbine_EOL_concreteAT)) * SRinert

GasTurbine_EOL_recycling_concreteCH = (RecyclingConcrete * RecyclingEfficiencyConcrete * GasTurbine_EOL_concreteCH) - (RecycledContentConcrete * GasTurbine_EOL_concreteCH)
GasTurbine_EOL_primary_concreteCH = ((RecycledContentConcrete * GasTurbine_EOL_concreteCH) - (RecyclingConcrete * RecyclingEfficiencyConcrete * GasTurbine_EOL_concreteCH)) * SRinert

GasTurbine_EOL_recycling_copper = (RecyclingMetals * RecyclingEfficiencyMetals * GasTurbine_EOL_copper) - (RecycledContentMetals * GasTurbine_EOL_copper)
GasTurbine_EOL_primary_copper = ((RecycledContentMetals * GasTurbine_EOL_copper) - (RecyclingMetals * RecyclingEfficiencyMetals * GasTurbine_EOL_copper)) * SRmetals

GasTurbine_EOL_recycling_HDPE = (RecyclingPEHD * RecyclingEfficiencyPlastics * GasTurbine_EOL_PEHD) - (RecycledContentPlastic * GasTurbine_EOL_PEHD)
GasTurbine_EOL_primary_HDPE = ((RecycledContentPlastic * GasTurbine_EOL_PEHD) - (RecyclingPEHD * RecyclingEfficiencyPlastics * GasTurbine_EOL_PEHD)) * SRinert

GasTurbine_EOL_recycling_reinforcingsteel = (RecyclingMetals * RecyclingEfficiencyMetals * GasTurbine_EOL_reinforcingsteel) - (RecycledContentMetals * GasTurbine_EOL_reinforcingsteel)
GasTurbine_EOL_primary_reinforcingsteel = ((RecycledContentMetals * GasTurbine_EOL_reinforcingsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * GasTurbine_EOL_reinforcingsteel)) * SRmetals

GasTurbine_EOL_recycling_chromiumsteel = (RecyclingMetals2 * RecyclingEfficiencyMetals * GasTurbine_EOL_chromiumsteel) - (RecycledContentMetals * GasTurbine_EOL_chromiumsteel)
GasTurbine_EOL_primary_chromiumsteel = ((RecycledContentMetals * GasTurbine_EOL_chromiumsteel) - (RecyclingMetals2 * RecyclingEfficiencyMetals * GasTurbine_EOL_chromiumsteel)) * SRmetals

## 3.10 Module D3 (related to the export of energy as result of waste incineration)

In [20]:
GasTurbine_EOL_energyrecoveryheat_PEHD = - GasTurbine_EOL_inc_PEHD * LHVHDPE * Xincheat
GasTurbine_EOL_energyrecoveryele_PEHD = - GasTurbine_EOL_inc_PEHD * LHVHDPE * Xincele

# 4. Data for export

In [21]:
ThermalEnergyProduced_GasTurbine = np.array(TotQ)